In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
%matplotlib widget
import numpy as np
import k3d
import matplotlib.pyplot as plt
from functools import reduce
from raytracing import *
from ipywidgets import *

In [9]:
def SixMirror(dx=27.77, dy=8.0, dz=16.685, d=4.750, dzF=1.5825, Rfast=25.0, eps=np.zeros((6,5))):

    p5 = np.array([0,0,0])
    p6 = np.array([dx, dy, dzF])
    p1 = np.array([0, dy, dzF])
    p2 = np.array([dx, 2*dy, 0])
    p3 = np.array([d, dy+d, dz])
    p4 = np.array([dx-d, dy-d, dz])
    
    ps = np.stack([p1,p2,p3,p4,p5,p6], axis=0)
    geom = geometry(ps)
    ns = geom['refl']
    ps = geom['mir']
    #add perturbations
    ps += eps[:,:3]
    axx = geom['xin']
    axy = 0.5*(geom['yin']+geom['yout'])
    ns = [rot_vec(rot_vec(ns[i], axx[i], np.deg2rad(eps[i,3])), axy[i], np.deg2rad(eps[i,4])) for i, n in enumerate(ns)]
    
    hi = 12.7
    qi=7.75
    #reference plane is expected between first and last element!
    elements = [CurvedMirror(p=ps[0], n=ns[0], diam=qi, R=Rfast, curv='CC'),\
                Mirror(p=ps[1], n=ns[1], diam=qi),\
                Mirror(p=ps[2], n=ns[2], diam=hi),\
                Mirror(p=ps[3], n=ns[3], diam=hi),\
                Mirror(p=ps[4], n=ns[4], diam=qi),\
                CurvedMirror(p=ps[5], n=ns[5], diam=qi, R=Rfast, curv='CC')]
    return elements, geom

In [13]:
def find_eigenray_formpe(elements, ray0, lr = 0.05, maxiter=500, tol=1e-5, Nrt=1, debug=False, clip=True):
    rcur = ray0.copy()
    trajs = []
    tols = []
    for i in range(maxiter):
        traj = propagate_system(elements, rcur, Nrt=Nrt, clip=clip)
        rnew = traj[-1,...]
        if np.isnan(rnew).any():
            if debug:
                print("Failed")
            break
        trajs.append(rnew)
        res = np.max(np.abs(rcur.flatten() - rnew.flatten()))
        #res = np.mean(np.abs(rcur.flatten() - rnew.flatten()))
        tols.append(res)
        rcur = (1.-lr)*rcur + lr*rnew
        #renormalize normal vector
        nnorm = np.linalg.norm(rcur[1,:,:], axis=-1)
        rcur[1,:,:] *= (1/nnorm)[:,None]
        if res<tol:
            break
    if debug:
        print("Finished in {} steps, reached tol {:.3e}".format(i, res))
            
    return rcur, np.stack(trajs, axis=0), np.array(tols)

In [14]:
elements, geom = SixMirror()

x0 = 0.5*(elements[0].p + elements[-1].p)
n0 = norm(elements[0].p - elements[-1].p)
rs = ray_bundle(p=x0, n=n0, n_radii=5, n_angles=2, R=0.2)

screen = Screen(p=x0, n=-n0, diam=7.75)
elements.append(screen)

In [15]:
#r0 = x0[None,:] + np.array([[0,0.2,0], [0,0,0.2], [0,0.2,0.2], [0,-0.2,-0.2]])
#s0 = np.broadcast_to(n0, r0.shape)
r0, s0 = x0, norm(n0 + np.array([0,0.2,0]))
ray0 = np.stack([np.atleast_2d(r0), np.atleast_2d(s0)], axis=0)

In [16]:
rconv, rseq, tols = find_eigenray_formpe(elements, ray0, lr=0.03, debug=True)

Finished in 499 steps, reached tol 5.637e-05


In [17]:
plt.figure()
plt.semilogy(np.arange(len(tols)), tols)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
def MPE(x):
    #this implementation assumes x of the form [n x k-1] where n is the spatial dimension
    U = x[:,1:-1] - x[:,:-2]
    c = -np.linalg.pinv(U)@(x[:,-1]-x[:,-2])
    c = np.append(c, 1.0)
    s = x[:,1:]@c/np.sum(c)
    return s

In [19]:
lr = 0.05
_, _, tols = find_eigenray_formpe(elements, ray0, lr=lr, debug=True)
rconv, allseq, alltols = find_eigenray_mpe_debug(elements, ray0, Niter=20, Nmpe=10, lr=lr)

Finished in 371 steps, reached tol 9.903e-06
Finished in 19 steps, reached tol 1.073e-01
Finished in 19 steps, reached tol 1.901e-02
Finished in 19 steps, reached tol 1.986e-04
Finished in 19 steps, reached tol 3.749e-07
Finished in 0 steps, reached tol 3.871e-11


In [20]:
plt.figure()
plt.semilogy(np.arange(len(tols)), tols)
plt.semilogy(np.arange(len(alltols)), alltols)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …